In [ ]:
from math import floor
import math 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

In [ ]:
"En este apartado se obtiene los bbordes por medio de Laplace "
# función para aplicar una convolución 2D a una imagen
def convolucion_2d(imagen, kernel):
    alto, ancho,= imagen.shape
    k_alto, k_ancho = kernel.shape
        # cálculo del borde para evitar desbordamiento
    bordeH = int((k_alto-1)/2)
    bordeW = int((k_ancho-1)/2)
    
    imagen_ampliada = np.zeros((alto + bordeH*2, ancho + bordeW*2))
    imagen_ampliada[bordeH:-bordeH, bordeW:-bordeW] = imagen
    resultado = np.zeros_like(imagen)
    
    # aplicación del kernel a cada píxel de la imagen
    for i in range(bordeH, alto+bordeH):
        for j in range(bordeW, ancho+bordeW):
            trozo = imagen_ampliada[i-bordeH:i+bordeH+1, j-bordeW:j+bordeW+1]
            resultado[i-bordeH, j-bordeW] = (trozo * kernel).sum()
    
    return resultado

# función para aplicar una umbralización a una imagen
def umbralizacion(imagen, umbral):
    resultado = np.zeros_like(imagen)
    resultado[imagen > umbral] = 255

    return resultado

def thresholdOtsu(img): #min(weightB*VarianceB + weightF*VarianceF)
    #Histogram
    hist = np.zeros(256)
    for row in range(img.shape[0]):
        for column in range(img.shape[1]):
            hist[img[row][column][0]] += 1
    
    pixelesInImg = img.shape[0]*img.shape[1]
    WCVList = []

    #Algorithm
    for threshold in range(len(hist)):
        #b where hist < threshold
        b = hist[:threshold]
        #Weight b
        w_b = sum(b)/(pixelesInImg)
        #Mean b
        m = 0
        for i in range(threshold):
            m += (i*hist[i])
        if(sum(b)==0):
            m_b = 0
        else:
            m_b = m/sum(b)
        #Variance b
        v = 0
        for i in range(threshold):
            v += ((i-m_b)**2) * hist[i]
        if(sum(b)==0):
            v_b = 0
        else:
            v_b = v/sum(b)

        #f where hist >= threshold
        f = hist[threshold:]
        #Weight f
        w_f = sum(f)/(pixelesInImg)
        #Mean f
        m = 0
        for i in range(threshold, len(hist)):
            m += (i*hist[i])
        m_f = m/sum(f)
        #Variance f
        v = 0
        for i in range(threshold, len(hist)):
            v += ((i-m_f)**2) * hist[i]
        v_f = v/sum(f)
        #Within_class_variance
        WCV = (w_b*v_b)+(w_f*v_f)
        WCVList.append(WCV)
        
    t = WCVList.index(min(WCVList))
    
    return t #The rigth threshold

# definición del kernel para la convolución
kernel = np.array([[-1, -1, -1], 
                   [-1,  8, -1], 
                   [-1, -1, -1]])

def factorial(x):
    return 1 if x == 0 else x * factorial(x - 1)

def triangle(n):
    return [[factorial(i) / (factorial(j) * factorial(i - j)) for j in range(i + 1)] for i in range(n)]


def kernelGauss(n):
    h = triangle(n)[n-1]
    v = np.vstack(h)
    #print(h*v)
    return h*v

def kerLaplace():
    return np.array([[-1, -1, -1],[-1, 8, -1],[-1, -1, -1]])

'''def KerGauss():
    return np.array([[1,2,4,2,1],
                    [2,4,8,2,4],
                    [1,2,4,4,1]])'''

def kerGauss():
    return np.ones((5,5))

def kernelGaussX(n):
    kernelGaussX = kernelGauss(n)
    kernelGaussX[:,0] = kernelGaussX[:,0] * -1
    kernelGaussX[:,1] = kernelGaussX[:,1] * 0
    return kernelGaussX

def kernelGaussY(n):
    kernelGaussY = kernelGauss(n)
    kernelGaussY[0, :] = kernelGaussY[0, :] * -1
    kernelGaussY[1, :] = kernelGaussY[1, :] * 0
    return kernelGaussY

In [ ]:
'''Esta permite obtener el umbral promedio de cada imagen '''
def thresholdOtsuGris(img):
    img = img.astype(np.uint8)
    # Calcular el histograma
    hist = np.zeros(256, dtype=np.uint8)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            hist[img[i, j]] += 1
    
    # Total de píxeles en la imagen
    total_pixels = img.shape[0] * img.shape[1]
    
    # Inicializar variables
    sum_intensity = 0
    sum_background = 0
    weight_background = 0
    best_threshold = 0
    max_between_variance = 0
    
    # Calcular el umbral óptimo de Otsu
    for threshold in range(256):
        weight_background += hist[threshold]
        weight_foreground = total_pixels - weight_background
        
        if weight_background == 0 or weight_foreground == 0:
            continue
        
        sum_intensity += threshold * hist[threshold]
        mean_background = sum_intensity / weight_background
        mean_foreground = (sum_intensity - mean_background * weight_background) / weight_foreground
        
        between_variance = weight_background * weight_foreground * (mean_background - mean_foreground) ** 2
        
        if between_variance > max_between_variance:
            max_between_variance = between_variance
            best_threshold = threshold
    
    return best_threshold


In [ ]:
def laplace(imagen):
    umbral = thresholdOtsu (np.array(imagen))
    matrixRes=(convolucion_2d(imagen.mean(axis=2),kerLaplace()))
    matrixRes = umbralizacion(matrixRes, umbral)
    return matrixRes.astype(np.uint8)

def suavizado (imagen):
    matrixRes = convolucion_2d(imagen,kerGauss())
    return matrixRes

def sobel(imagen, n):
    umbral = thresholdOtsuGris(np.array(imagen))
    matrixRes = np.zeros((imagen.shape[0], imagen.shape[1]))
    matrixResX=(convolucion_2d(imagen.mean(axis=2),kernelGaussX(n) ))
    matrixResY=(convolucion_2d(imagen.mean(axis=2),kernelGaussX(n) ))
    

    for x in range(imagen.shape[0]):
        for y in range(imagen.shape[1]):
            matrixRes[x][y] = math.sqrt(matrixResX[x][y]**2 + matrixResY[x][y] **2)
    #muestra de la capa X y la capa Y
    matrixRes = umbralizacion(matrixRes, umbral)
    
    return matrixRes.astype(np.uint8)

def perfilado(imagen,n,a):
    umbral = thresholdOtsu(np.array(imagen))
    kernelPerf =np.array([[0,  -a,       0],
                          [-a, (n*a)+1, -a], 
                          [0,  -a,       0]])  
    matrixRes = convolucion_2d(imagen.mean(axis=2), kernelPerf)
    matrixRes = umbralizacion(matrixRes, 255-umbral)
    
    return matrixRes.astype(np.uint8)


In [ ]:
#Elimiinar el ruido de la imagen

#Expande las areas blancas
def dilatacion(A,B):
    m, n = A.shape
    p, q = B.shape
    C = np.zeros((m, n))

    for i in range(m):
        for j in range(n):
            for k in range(p):
                for l in range(q):
                    if B[k, l] == 1 and i+k-p//2 >= 0 and i+k-p//2 < m and j+l-q//2 >= 0 and j+l-q//2 < n:
                        C[i, j] = np.max([C[i, j], np.max(A[i+k-p//2, j+l-q//2])])
                        

    return C

#Se expaden las areas negras 
def erosion(imagen, kernel):
    altura, ancho = imagen.shape
    m, n = kernel.shape
    resultado = np.zeros((altura, ancho))
    
    for i in range(m//2, altura-m//2):
        for j in range(n//2, ancho-n//2):
            # Realizar la operación de erosión
            if (imagen[i-m//2:i+m//2+1, j-n//2:j+n//2+1] * kernel).min() >= 1:
                resultado[i, j] = 1
    
    return resultado

def abierto(A, B):
    C = erosion(A, B)
    D = dilatacion(C, B)
    return D

def cerrado(A, B):
    C = dilatacion(A, B)
    D = erosion(C, B)
    return D

In [ ]:
def abrirArchivo():
    ruta_carpeta = 'C:\ProyectoImgIa\ProyectoFinalTumor\yes'

    # Obtener la lista de archivos en la carpeta
    archivos = os.listdir('yes/')

    # Iterar sobre cada archivo en la carpeta
    for archivo in archivos:
        # Comprobar si el archivo es una imagen
            # Ruta completa de la imagen
            ruta_imagen = os.path.join('yes/', archivo)

            # Abrir la imagen utilizando PIL/Pillow
            imagen = Image.open(ruta_imagen)
            print(ruta_imagen)
            if imagen.mode != 'RGB':
                # Convertir la imagen a RGB si no tiene 3 canales
                imagen = imagen.convert('RGB')
            imagen_np = np.array(imagen)
            imagen_gris = np.mean(imagen_np, axis=2).astype(np.uint8)
            #imagen = recorte(np.array(imagen), thresholdOtsuGris(np.array(imagen))).astype(np.uint8)
            bordeLA = laplace(imagen_np)
            bordeSo = sobel(imagen_np,3)
            perfil = perfilado(imagen_np,4,1)

            fig, axs = plt.subplots(1,3,figsize=(20, 5))
            axs[0].imshow(bordeSo, cmap ="gray")
            axs[0].set_title("Sobel bordering")
            axs[1].imshow(bordeLA, cmap ="gray")
            axs[1].set_title("Laplace bordering")
            axs[2].imshow(perfil, cmap ="gray")
            plt.show()

abrirArchivo()